![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [191]:
import pandas as pd
import numpy as np

# Reading in and splitting the data

In [192]:
# Read in bank_marketing.csv as a pandas DataFrame
df_bm = pd.read_csv('bank_marketing.csv')
df_bm.sample(10)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
30137,30137,39,technician,married,professional.course,no,yes,no,cellular,apr,24,67,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no
37898,37898,38,admin.,single,university.degree,no,yes,no,telephone,sep,29,490,1,999,0,nonexistent,-3.4,92.379,-29.8,0.819,5017.5,no
18109,18109,46,admin.,married,university.degree,no,no,no,cellular,jul,22,29,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
30699,30699,36,blue-collar,married,basic.9y,no,no,no,cellular,may,24,151,1,999,0,nonexistent,-1.8,92.893,-46.2,1.344,5099.1,no
41145,41145,26,admin.,single,university.degree,no,no,no,cellular,nov,23,528,3,999,0,nonexistent,-1.1,94.767,-50.8,1.040,4963.6,yes
40104,40104,19,student,single,basic.9y,no,no,yes,telephone,jul,7,567,1,6,2,success,-1.7,94.215,-40.3,0.827,4991.6,yes
181,181,46,blue-collar,married,basic.6y,unknown,yes,no,telephone,may,31,240,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3847,3847,45,management,married,university.degree,no,no,no,telephone,may,20,255,3,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0,no
4733,4733,35,retired,married,professional.course,no,no,no,telephone,may,18,759,1,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,no
23146,23146,30,technician,divorced,professional.course,unknown,yes,no,cellular,aug,23,210,3,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,no


In [193]:
df_bm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [194]:
df_bm.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

### Split the data into three DataFrames that contain information from desired tables

In [195]:
client = df_bm[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan',]]
campaign = df_bm[['client_id',  'duration', 'pdays', 'previous', 'y', 'poutcome', 'campaign']]
economics =df_bm[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

###  Rename the columns from the three dataframes 

In [196]:
client = client.rename(columns = {'client_id': 'id'})
print(client.columns)

campaign = campaign.rename(columns ={"duration":"contact_duration", "previous": "previous_campaign_contacts", 'y': "campaign_outcome", "poutcome": "previous_outcome", "campaign": "number_contacts"})
print(campaign.columns)

economics = economics.rename(columns = {"euribor3m":"euribor_three_months", "nr_employed":"number_employed"})
print(economics.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['client_id', 'contact_duration', 'pdays', 'previous_campaign_contacts',
       'campaign_outcome', 'previous_outcome', 'number_contacts'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


# Cleaning the data

###  Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values

In [197]:
client['education'] = client['education'].str.replace(".", "_")
client['education'] = client['education'].str.replace("unknown", 'np.nan')
print(client['education'].isna().sum())
client['education'].value_counts()

0


university_degree      12168
high_school             9515
basic_9y                6045
professional_course     5243
basic_4y                4176
basic_6y                2292
np.nan                  1731
illiterate                18
Name: education, dtype: int64

### Remove periods  from the "job" column.

In [198]:
client['job'] = client['job'].str.replace('.','')
client['job'].value_counts()

admin            10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

### Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome"

In [199]:
print(campaign['previous_outcome'].value_counts())
campaign['campaign_outcome'].value_counts()

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64


no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [200]:
mapping= { "success": 1, "failure": 0, "nonexistent": np.nan}
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace(mapping)
print(campaign['campaign_outcome'].isna().sum())
campaign['campaign_outcome'].value_counts()

0


no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [201]:
campaign['previous_outcome'] = campaign['previous_outcome'].replace(mapping)
print(campaign['previous_outcome'].isna().sum())
campaign['previous_outcome'].value_counts()

35563


0.0    4252
1.0    1373
Name: previous_outcome, dtype: int64

### Add a column called campaign_id in campaign, where all rows have a value of 1

In [202]:
campaign['campaign_id'] = 1
campaign['campaign_id'].sample(5)

30755    1
27188    1
838      1
271      1
967      1
Name: campaign_id, dtype: int64

### Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.

In [203]:
campaign['last_contact_date'] = '2022' + '-'+ df_bm['month'].str.capitalize() + '-' + df_bm['day'].astype('str')
campaign['last_contact_date'].sample(5)

33816    2022-May-23
12808    2022-Jul-30
1844      2022-May-2
20772     2022-Aug-7
31734    2022-May-22
Name: last_contact_date, dtype: object

In [204]:
pd.to_datetime(campaign['last_contact_date'], format = '%Y-%b-%d') 

0       2022-05-13
1       2022-05-19
2       2022-05-23
3       2022-05-27
4       2022-05-03
           ...    
41183   2022-11-30
41184   2022-11-06
41185   2022-11-24
41186   2022-11-17
41187   2022-11-23
Name: last_contact_date, Length: 41188, dtype: datetime64[ns]

### Reorder the columns, ensuring the columns in each subset of the data match the table displayed in the notebook.

In [205]:
campaign_order = ['campaign_id','client_id', 'number_contacts', 'contact_duration', 'pdays', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date']
campaign = campaign.reindex(campaign_order, axis = 1)
campaign.columns

Index(['campaign_id', 'client_id', 'number_contacts', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'last_contact_date'],
      dtype='object')

# Saving the data

### Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.

In [206]:
client.to_csv('client.csv', index = False)
campaign.to_csv('campaign.csv', index = False)
economics.to_csv('economics.csv', index = False)
!ls

bank_marketing.csv  client.csv	   notebook.ipynb
campaign.csv	    economics.csv  piggy_bank.jpg


# Designing the database

In [207]:
# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [208]:
campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [209]:
economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""